In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
data_dict = pd.read_csv("Data/data_dictionary.csv")
products = pd.read_csv("Data/products.csv")
accounts = pd.read_csv("Data/accounts.csv")
sales_pipeline = pd.read_csv("Data/sales_pipeline.csv")
sales_teams = pd.read_csv("Data/sales_teams.csv")

# Data cleaning 

In [3]:
data_dict

,Table,Field,Description
0,accounts,account,Company name
1,accounts,sector,Industry
2,accounts,year_established,Year Established
3,accounts,revenue,Annual revenue (in millions of USD)
4,accounts,employees,Number of employees
5,accounts,office_location,Headquarters
6,accounts,subsidiary_of,Parent company
7,products,product,Product name
8,products,series,Product series
9,products,sales_price,Suggested retail price


In [4]:
accounts.head()

,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,Acme Corporation,technolgy,1996,1100.04,2822,United States,NaN
1,Betasoloin,medical,1999,251.41,495,United States,NaN
2,Betatech,medical,1986,647.18,1185,Kenya,NaN
3,Bioholding,medical,2012,587.34,1356,Philipines,NaN
4,Bioplex,medical,1991,326.82,1016,United States,NaN


## Convert Date columns into DataTime Datatype

In [5]:
sales_pipeline.engage_date = pd.to_datetime(sales_pipeline.engage_date)

In [6]:
sales_pipeline.close_date = pd.to_datetime(sales_pipeline.close_date)

In [7]:
pipeline = sales_pipeline.copy()

In [8]:
pipeline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   opportunity_id  8800 non-null   object        
 1   sales_agent     8800 non-null   object        
 2   product         8800 non-null   object        
 3   account         7375 non-null   object        
 4   deal_stage      8800 non-null   object        
 5   engage_date     8300 non-null   datetime64[ns]
 6   close_date      6711 non-null   datetime64[ns]
 7   close_value     6711 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 550.1+ KB


## Create columns

- enagege_quarter - the quarter period the engagement date falls into (Fiscal year begins in January)
- close_quarter - the quarter period the close date falls into (Fiscal year begins in January)
- sale_duration - the ammount of time in days to close a deal
- close_fiscal_year - the fiscal year the deal was closed
- close_quarter_tags - the quarter when the deal was closed (shows the month the quarter ends)
- quarter_tags - shows quarter the deal was closed (Q1, Q2, Q3 or Q4)

In [9]:
pipeline["engage_quarter"] = pipeline.engage_date.dt.to_period("Q-DEC")
pipeline["close_quarter"] = pipeline.close_date.dt.to_period("Q-DEC")

In [10]:
pipeline["sale_duration"] = pipeline.close_date - pipeline.engage_date

In [11]:
pipeline.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,engage_quarter,close_quarter,sale_duration
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,2016Q4,2017Q1,132 days
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0,2016Q4,2017Q1,137 days
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0,2016Q4,2017Q1,133 days
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,2016Q4,2017Q1,135 days
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Won,2016-10-25,2017-03-02,517.0,2016Q4,2017Q1,128 days


In [12]:
sales_teams.head()

,sales_agent,manager,regional_office
0,Anna Snelling,Dustin Brinkmann,Central
1,Cecily Lampkin,Dustin Brinkmann,Central
2,Versie Hillebrand,Dustin Brinkmann,Central
3,Lajuana Vencill,Dustin Brinkmann,Central
4,Moses Frase,Dustin Brinkmann,Central


### Merge 2 datasets (the sales_pipeline and sales_teams datasets)

In [13]:
sales_pipeline_teams = pipeline.merge(
    sales_teams,
    "inner",
    on="sales_agent"
)

In [14]:
sales_pipeline_teams.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,engage_quarter,close_quarter,sale_duration,manager,regional_office
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,2016Q4,2017Q1,132 days,Dustin Brinkmann,Central
1,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,2016Q4,2017Q1,135 days,Dustin Brinkmann,Central
2,WF4HA5NW,Moses Frase,MG Special,Ron-tech,Won,2016-11-07,2017-03-18,50.0,2016Q4,2017Q1,131 days,Dustin Brinkmann,Central
3,LGI8QEC2,Moses Frase,GTX Basic,Treequote,Won,2016-12-02,2017-03-06,583.0,2016Q4,2017Q1,94 days,Dustin Brinkmann,Central
4,I043RXJV,Moses Frase,GTX Plus Basic,Dontechi,Lost,2016-12-03,2017-03-17,0.0,2016Q4,2017Q1,104 days,Dustin Brinkmann,Central


In [15]:
sales_pipeline_teams["close_fiscal_year"] = sales_pipeline_teams.close_quarter.dt.qyear

In [16]:
sales_pipeline_teams["close_quarter_tags"] = pd.PeriodIndex(sales_pipeline_teams["close_quarter"], freq="Q").strftime("%m-%Y")

In [17]:
sales_pipeline_teams["quarter_tags"] = sales_pipeline_teams.close_quarter.astype(str).replace("NaT", np.nan).str.slice(-2)

### Corrected an entry for products
- was initially GTXPro to GTX Pro

In [18]:
sales_pipeline_teams["product"] = sales_pipeline_teams["product"].replace("GTXPro", "GTX Pro")

In [19]:
sales_pipeline_teams.head()

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,engage_quarter,close_quarter,sale_duration,manager,regional_office,close_fiscal_year,close_quarter_tags,quarter_tags
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0,2016Q4,2017Q1,132 days,Dustin Brinkmann,Central,2017,03-2017,Q1
1,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Won,2016-10-25,2017-03-09,588.0,2016Q4,2017Q1,135 days,Dustin Brinkmann,Central,2017,03-2017,Q1
2,WF4HA5NW,Moses Frase,MG Special,Ron-tech,Won,2016-11-07,2017-03-18,50.0,2016Q4,2017Q1,131 days,Dustin Brinkmann,Central,2017,03-2017,Q1
3,LGI8QEC2,Moses Frase,GTX Basic,Treequote,Won,2016-12-02,2017-03-06,583.0,2016Q4,2017Q1,94 days,Dustin Brinkmann,Central,2017,03-2017,Q1
4,I043RXJV,Moses Frase,GTX Plus Basic,Dontechi,Lost,2016-12-03,2017-03-17,0.0,2016Q4,2017Q1,104 days,Dustin Brinkmann,Central,2017,03-2017,Q1


# Create .pickle file 

In [20]:
# pd.to_pickle(sales_pipeline_teams, "Data/sales_pipeline_and_teams.pkl")

# Dash App

In [ ]:
# %%writefile mavin_sales_app.py

from dash import Dash, dcc, html
import pandas as pd
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

from dash.dependencies import Input, Output

import plotly.express as px

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

sales = pd.read_pickle("Data/sales_pipeline_and_teams.pkl")
sales_teams = pd.read_csv("Data/sales_teams.csv")
products = pd.read_csv("Data/products.csv")



app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SOLAR")

app.layout = dbc.Container([
    dbc.Row([
        html.H1(id = "header_tag", style={"textAlign": "center"})
    ]),
    html.Hr(),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                html.P("Choose Manager", style={"textAlign": "center"}),
                dcc.Dropdown(
                    id="manager_tag",
                    options=list(sales_teams.manager.unique()),
                    value="",
                ),
                html.Br(),
                html.Hr(),
                html.P("Select Year to Review", style={"textAlign": "center"}),
                dcc.Dropdown(
                    id="year_tag",
                    options=list(sales.close_fiscal_year.value_counts().sort_index().index)[1:],
                    value="2017"
                ),
                html.Br(),
                html.Hr(),
                html.P("Select Quarter", style={"textAlign": "center"}),
                dcc.RadioItems(
                    id="quarter_tag",
                    options= list(sales.quarter_tags.unique())[:4],
                    value="Q1",
                ),
                html.Hr(),
                html.P("Only for the Bar Plots", style={"textAlign": "center"}),
                dcc.RadioItems(
                    id="top_bottom",
                    options=["top to bottom", "bottom to top"],
                    value="top to bottom"
                )
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([html.H3("Revenue Generated in Quarter", style={"textAlign": "center"})], className="text-title"),
                dcc.Graph(id="pie_revenue"),
            ])
        ], width=5),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3("Top Sold Product by Units", style={"textAlign": "center"}),
                    html.Br(),
                    html.H5(id="product_tag", style={"textAlign": "center"}),
                    html.H5(id="product_price", style={"textAlign": "center"}),
                    html.Br(),
                    html.Hr(),
                    html.H3("Top Revenue Generating Product", style={"textAlign": "center"}),
                    html.Br(),
                    html.H5(id="top_revenue_product", style={"textAlign": "center"}),
                    html.H5(id="product_revenue", style={"textAlign": "center"}),
                    html.Br(),
                    html.Hr(),
                    html.H3("Top Revenue Generating Client", style={"textAlign": "center"}),
                    html.Br(),
                    html.H5(id="top_client", style={"textAlign": "center"}),
                    html.Br(),
                ])
            ])
        ], width=5)
    ]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3("Quarter on Quarter Revenue", style={"textAlign": "center"})
                ]),
                dcc.Graph(id="quarter_revenue_line")
            ])
        ], width=4),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3("Closers who 'Won' Deals", style={"textAlign": "center"})
                ]),
                dcc.Graph(id="top_closes_bar")
            ])
        ], width=4),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H3("Revenue Generated by Agent", style={"textAlign": "center"})
                ]),
                dcc.Graph(id="top_revenue_bar")
            ])
        ], width=4)
    ]),
])


@app.callback(
    Output("pie_revenue", "figure"),
    Input("manager_tag", "value"),
    Input("year_tag", "value"),
    Input("quarter_tag", "value")
)

def donut_chart(manager, close_year, close_quarter):
    if not manager:
        raise PreventUpdate
    if not close_year:
        raise PreventUpdate
    
    closing_year = int(pd.to_numeric(close_year))
    
    manager_df = sales.query("manager == @manager").query("close_fiscal_year == @closing_year").query("quarter_tags == @close_quarter").reset_index(drop=True)
    manager_name = manager_df.loc[0, "manager"]
    manager_revenue = manager_df.close_value.sum()
    other_managers_revenue = sales.query("close_fiscal_year == @closing_year").query("quarter_tags == @close_quarter").close_value.sum() - manager_revenue

    fig = px.pie(
        values=[manager_revenue, other_managers_revenue],
        names=[manager_name, "Other Managers"],
        hole=0.6,
    )

    return fig

@app.callback(
    Output("header_tag", "children"),
    Output("product_tag", "children"),
    Output("product_price", "children"),
    Output("top_revenue_product", "children"),
    Output("product_revenue", "children"),
    Output("top_client", "children"),
    Input("manager_tag", "value"),
    Input("year_tag", "value"),
    Input("quarter_tag", "value"),
)

def texts_update(manager, year_close, close_quarter):
    if not manager:
        raise PreventUpdate
    if not year_close:
        raise PreventUpdate
    
    header_title = f"Sales Data Analysis by Manager {manager}"
    
    closing_year = int(pd.to_numeric(year_close))
    df = sales.query("manager == @manager").query("close_fiscal_year == @closing_year").query("quarter_tags == @close_quarter").query("deal_stage == 'Won'")

    top_product = df.groupby("product").agg({"opportunity_id": "count"}).sort_values(by="opportunity_id", ascending=False).reset_index().loc[0,"product"]
    product_price = f'Unit price of USD$ {products.groupby("product").agg({"sales_price": "sum"}).loc[top_product, "sales_price"]:,.0f}'
    top_revenue_product = df.groupby("product").agg({"close_value": "sum"}).sort_values(by="close_value", ascending=False).reset_index().loc[0,"product"]
    product_revenue = f'USD$ {df.groupby("product").agg({"close_value": "sum"}).sort_values(by="close_value", ascending=False).reset_index().loc[0,"close_value"]:,.0f}'
    top_client = df.groupby("account").agg({"close_value": "sum"}).sort_values(by="close_value", ascending=False).reset_index().loc[0,"account"]

    return header_title, top_product, product_price, top_revenue_product, product_revenue, top_client

@app.callback(
    Output("top_closes_bar", "figure"),
    Output("top_revenue_bar", "figure"),
    Input("manager_tag", "value"),
    Input("year_tag", "value"),
    Input("quarter_tag", "value"),
    Input("top_bottom", "value"),
)

def bar_plots_update(manager, year_close, close_quarter, groupings):
    if not manager:
        raise PreventUpdate
    if not year_close:
        raise PreventUpdate
    
    closing_year = int(pd.to_numeric(year_close))

    df = sales.query("manager == @manager").query("close_fiscal_year == @closing_year").query("quarter_tags == @close_quarter").query("deal_stage == 'Won'")

    top_closers = df.groupby("sales_agent").agg({"account": "count"}).sort_values(by="account", ascending=False).reset_index()

    bottom_closers = df.groupby("sales_agent").agg({"account": "count"}).sort_values(by="account").reset_index()
    
    top_revenue_gen = df.groupby("sales_agent").agg({"close_value": "sum"}).sort_values(by="close_value", ascending=False).reset_index()

    bottom_revenue_gen = df.groupby("sales_agent").agg({"close_value": "sum"}).sort_values(by="close_value").reset_index()

    if groupings == "top to bottom":
        fig = px.bar(top_closers,
                    x="sales_agent",
                    y="account",
                    labels={
                        "account": "Frequency",
                        "sales_agent": "Sales Agent"
                    }
                    ).update_layout(showlegend=False)

        fig1 = px.bar(top_revenue_gen,
                    x="sales_agent",
                    y="close_value",
                    labels={
                        "close_value": "Revenue",
                        "sales_agent": "Sales Agent"
                    }
                    ).update_layout(showlegend=False)
    else:
        fig = px.bar(bottom_closers,
                    x="sales_agent",
                    y="account",
                    labels={
                        "account": "Frequency",
                        "sales_agent": "Sales Agent"
                    }
                    ).update_layout(showlegend=False)

        fig1 = px.bar(bottom_revenue_gen,
                    x="sales_agent",
                    y="close_value",
                    labels={
                        "close_value": "Revenue",
                        "sales_agent": "Sales Agent"
                    }
                    ).update_layout(showlegend=False)

    return fig, fig1

@app.callback(
    Output("quarter_revenue_line", "figure"),
    Input("manager_tag", "value"),
)

def qoq_revenue_update(manager):
    if not manager:
        raise PreventUpdate
    
    df = sales.query("manager == @manager")

    fig = px.line(df.groupby("close_quarter_tags").agg({"close_value": "sum"}).sort_index().reset_index(),
                  x="close_quarter_tags",
                  y="close_value",
                  labels={
                      "close_value": "Revenue",
                      "close_quarter_tags": "Quarter"
                  },
                  ).update_layout(showlegend=False)

    return fig
    
if __name__ == "__main__":
    app.run_server(port=5000)